# 第２章 確率空間

参考：[ギリシャ文字等の入力方法](https://qiita.com/alchemist/items/0ce850770d8cc3df0ab4)

参考：[Pythonからはじめる数学入門](https://www.amazon.co.jp/dp/4873117682)

In [1]:
from sympy import *

## 2.1 見本空間 $\Omega$

### 見本

注意点：sympy の仕様なのか，要素を文字にすると思うような動作が得られないので，以下，整数に置き換えて計算します．

In [2]:
a=1
b=2
c=3

### 見本空間

In [3]:
Ω = FiniteSet(a,b,c) #\Omega[Tab] で入力可能
Ω

{1, 2, 3}

## 2.2 事象の代数 $\mathcal F$

## 2.2.1 事象

In [4]:
A1 = FiniteSet(a,c)
A1

{1, 3}

In [5]:
A2 = FiniteSet(b,c)
A2

{2, 3}

## 2.2.2 事象の代数

### 事象の族 F

In [6]:
F = FiniteSet(A1,A2)
F

{{1, 3}, {2, 3}}

### 和事象

In [7]:
A1 + A2

{1, 2, 3}

### 積事象

In [8]:
A1 & A2

{3}

### 余事象

In [9]:
Ω - A1

{2}

In [10]:
Ω - A2

{1}

### 式(2.5) --- 追加後の族　

In [11]:
F1 = F + FiniteSet( A1+A2, A1&A2, Ω-A1, Ω-A2 )
F1

{{1}, {2}, {3}, {1, 3}, {2, 3}, {1, 2, 3}}

### 式(2.6) --- 不足チェック

In [12]:
Ω-(A1+A2)

EmptySet()

In [13]:
Ω-(A1+A2) in F1

False

### 式(2.7) --- 完成

In [14]:
FF = F1 + FiniteSet(Ω-(A1+A2),Ω-(A1&A2))
FF

{EmptySet(), {1}, {2}, {3}, {1, 2}, {1, 3}, {2, 3}, {1, 2, 3}}

## 2.2.3 $\sigma$代数

### 定義2.1 --- チェック用の関数はこんな感じ？

In [15]:
def is_sigma_algebra(Ω,FF):
    is_ok = True
    
    # (σ1)のチェック
    is_ok = is_ok and (Ω in FF)
    
    # for文で全要素を巡るためのリスト化
    elm=list(FF) #リスト化
    n=len(elm)   #リストの要素数
    
    # (σ2)のチェック
    for i in range(0,n):
        is_ok = is_ok and (Ω-elm[i] in FF)

    # (σ3)の全数チェック 
    for i in range(0,n):
        for j in range(i,n):
            is_ok = is_ok and (elm[i]+elm[j] in FF)
    return is_ok

In [16]:
is_sigma_algebra(Ω,F1)

False

In [17]:
is_sigma_algebra(Ω,FF)

True

### 生成用の関数ってこんな感じ？

#### 部品１： 補集合を追加する関数

In [18]:
def append_complements(Ω,F):
    result=F
    for elm in list(F):
        celm = Ω - elm
        result = result + FiniteSet(celm)
    return result

In [19]:
F

{{1, 3}, {2, 3}}

In [20]:
append_complements(Ω,F)

{{1}, {2}, {1, 3}, {2, 3}}

#### 部品2： 全ての組み合わせの和集合を追加する関数

In [21]:
def append_unions(F):
    result=F
    Flist=list(F)
    Fn=len(Flist)
    for i in range(0,Fn):
        for j in range(i,Fn):
            result = result + FiniteSet(Flist[i]+Flist[j])
    return result

In [22]:
append_unions(F)

{{1, 3}, {2, 3}, {1, 2, 3}}

#### $\sigma$代数（有限版）を生成する関数　（注）ここでの$\sigma$代数は要素数が有限なので，本当はブール代数という．

In [23]:
def generate_sigma_algebra(Ω,F):
    cur_F = F + FiniteSet(Ω)
    prev_F = EmptySet()
    while(prev_F != cur_F):
        prev_F = cur_F
        cur_F = append_complements(Ω,cur_F)
        cur_F = append_unions(cur_F)
    return cur_F

In [24]:
FFauto = generate_sigma_algebra(Ω,F)
FFauto

{EmptySet(), {1}, {2}, {3}, {1, 2}, {1, 3}, {2, 3}, {1, 2, 3}}

In [25]:
is_sigma_algebra(Ω,FFauto)

True

### 例題2.1

In [26]:
F_2_1=FiniteSet(FiniteSet(a,c),FiniteSet(b))
generate_sigma_algebra(Ω,F_2_1)

{EmptySet(), {2}, {1, 3}, {1, 2, 3}}

### 演習2.2

In [27]:
F_eg21=FiniteSet(FiniteSet(a,b),FiniteSet(b))
generate_sigma_algebra(Ω,F_eg21)

{EmptySet(), {1}, {2}, {3}, {1, 2}, {1, 3}, {2, 3}, {1, 2, 3}}

### おまけ --- その他の場合

In [28]:
generate_sigma_algebra(Ω,FiniteSet(FiniteSet(b)))

{EmptySet(), {2}, {1, 3}, {1, 2, 3}}

### 補足： 最大の$\sigma$代数 = $\Omega$の冪集合

In [29]:
Ω.powerset()

{EmptySet(), {1}, {2}, {3}, {1, 2}, {1, 3}, {2, 3}, {1, 2, 3}}

この問題のFで生成した$\sigma$代数は，$\Omega$の冪集合にたまたま一致する．そうでないのもある．例題2.1参照．

In [30]:
FF == Ω.powerset()

True

## 2.3 確率 $P$

## 2.3.1 見本に基づく方法

### 式(2.9)

In [31]:
def P(A):
    if A == FiniteSet(a):
        p = 1/3
    elif A == FiniteSet(b):
        p = 1/3
    elif A == FiniteSet(c):
        p = 1/3
    else:
        p = False
    return p

In [32]:
P(FiniteSet(a))

0.3333333333333333

### 確率を自動計算する関数

In [33]:
def Pauto(A):
    lst = list(A)
    n = len(lst)
    if n == 1:
        if A == FiniteSet(a):
            p = 1/3
        elif A == FiniteSet(b):
            p = 1/3
        elif A == FiniteSet(c):
            p = 1/3
        else:
            p = False
    elif n > 1:
        p = 0;
        for x in lst:
            p += Pauto(FiniteSet(x))
    return p

### 演習2.4

In [34]:
Pauto(A2)

0.6666666666666666

In [35]:
Pauto(Ω)

1.0

### おまけ --- 式(2.10)の確率を使った場合

In [36]:
def Pauto2(A):
    lst = list(A)
    n = len(lst)
    if n == 1:
        if A == FiniteSet(a):
            p = 1/4
        elif A == FiniteSet(b):
            p = 2/4
        elif A == FiniteSet(c):
            p = 1/4
        else:
            p = False
    elif n > 1:
        p = 0;
        for x in lst:
            p += Pauto2(FiniteSet(x))
    return p

In [37]:
Pauto2(A2)

0.75

In [38]:
Pauto2(Ω)

1.0

## 2.4 確率変数 $X(\omega)$

## 2.4.1 確率変数

### 式(2.16)

In [39]:
def X(ω):
    if ω == a:
        val = 3.5
    if ω == b:
        val = 2.8
    if ω == c:
        val = 3.5
    return val

In [40]:
def Y(ω):
    if ω == a:
        val = 22
    if ω == b:
        val = 22
    if ω == c:
        val = 25
    return val

In [41]:
X(a), X(b), X(c), Y(a), Y(b), Y(c)

(3.5, 2.8, 3.5, 22, 22, 25)

## 2.4.2 測定値の生起確率

### $X^{-1}$ を求める関数

In [42]:
def FUNCinv(func,Ω,val):
    invset = EmptySet()
    for ω in Ω:
        if func(ω) == val:
            invset += FiniteSet(ω)
    return invset

In [43]:
def Xinv(Ω,val):
    return FUNCinv(X,Ω,val)

In [44]:
Xinv(Ω, 3.5)

{1, 3}

In [45]:
Xinv(Ω, 2.8)

{2}

### 演習2.6 --- $Y^{-1}$ を求める関数

In [46]:
def Yinv(Ω,val):
    return FUNCinv(Y,Ω,val)

In [47]:
Yinv(Ω, 22)

{1, 2}

In [48]:
Yinv(Ω, 25)

{3}

### 演習2.7 --- 確率変数の値の確率

In [49]:
Pauto(Xinv(Ω, 3.5))

0.6666666666666666

In [50]:
Pauto(Xinv(Ω, 2.8))

0.3333333333333333

In [51]:
Pauto(Yinv(Ω, 22))

0.6666666666666666

In [52]:
Pauto(Yinv(Ω, 25))

0.3333333333333333

### おまけ --- 式(2.10)の確率を使った場合

In [53]:
Pauto2(Xinv(Ω, 3.5))

0.5

In [54]:
Pauto2(Xinv(Ω, 2.8))

0.5

In [55]:
Pauto2(Yinv(Ω, 22))

0.75

In [56]:
Pauto2(Yinv(Ω, 25))

0.25

## 2.4.3 確率変数から生成される$\sigma$代数

### 式(2.24), (2.25) --- 確率変数$X(\omega)$から生成される$\sigma$代数

In [57]:
generate_sigma_algebra(Ω, FiniteSet( Xinv(Ω, 2.8), Xinv(Ω, 3.5) ))

{EmptySet(), {2}, {1, 3}, {1, 2, 3}}

### 演習2.8 --- 確率変数$Y(\omega)$から生成される$\sigma$代数

In [58]:
generate_sigma_algebra(Ω, FiniteSet( Yinv(Ω, 22), Yinv(Ω, 25) ))

{EmptySet(), {3}, {1, 2}, {1, 2, 3}}

### 演習2.9 --- 超つまんない例　．．．足の数　申し訳ございません <(_"_)>

In [59]:
def Z(ω):
    if ω == a:
        val = 4
    if ω == b:
        val = 2
    if ω == c:
        val = 2
    return val

In [60]:
def Zinv(Ω,val):
    invset = EmptySet()
    for ω in Ω:
        if Z(ω) == val:
            invset += FiniteSet(ω)
    return invset

In [61]:
generate_sigma_algebra(Ω, FiniteSet( Zinv(Ω, 2), Zinv(Ω, 4) ))

{EmptySet(), {1}, {2, 3}, {1, 2, 3}}

今度は {2,3} = {b,c} が含まれているので，確率変数$Z$を採用すれば，二足歩行{b,c}は可測となる．

感想： 確率空間の計算，python で全部書けてしまった．すごいな python．もっとはやく使えばよかった・・・